<a href="https://colab.research.google.com/github/lmfandre/Desafio_Clustering/blob/main/RID164252_Desafio07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as mp

In [ ]:
df = pd.read_csv("data (1).csv", encoding="ISO-8859-1")
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Description e CustomerID contêm dados nulos

In [ ]:
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


Temos 135.080 dados faltantes de clientes. São muitos dados, mas devem ser dropados para calcular o RFM

In [ ]:
df = df.dropna(subset=['CustomerID'])

In [ ]:
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

Valores negativos em Quantity e UnitPrice não fazem sentido

In [ ]:
menorq0 = df.query("Quantity <= 0 or UnitPrice <= 0")
menorq0

In [ ]:
df = df.query("Quantity > 0 and UnitPrice > 0").copy()
df

In [ ]:
df[["Quantity", "UnitPrice"]].plot.box()

Existem dois valores outliers muito gritantes em Quantity, pode ser interessante ignorá-los na nossa análise


In [ ]:
df.query("Quantity > 10000")

In [ ]:
df.query("Quantity < 10000")[["Quantity", "UnitPrice"]].plot.box()

In [ ]:
df.query("UnitPrice > 8000")

Também vamos eliminar este outlier

In [ ]:
df = df.query("Quantity < 10000 & UnitPrice < 8000").copy()

In [ ]:
df["TotalPrice"] = df.Quantity * df.UnitPrice
df

In [ ]:
df[["Description", "Country"]].nunique()

In [ ]:
df["InvoiceDate"] = pd.to_datetime(df.InvoiceDate, format = "%m/%d/%Y %H:%M")

In [ ]:
df["CustomerID"] = df["CustomerID"].astype(int)
df["Country"] = df["Country"].astype("category")

In [ ]:
df.info()

In [ ]:
# **Agrupar por Cliente e Pedido (InvoiceNo)**
df_pedidos = df.groupby(["CustomerID", "InvoiceNo"]).agg(
    InvoiceDate=("InvoiceDate", "max"),  # Pega a data do pedido
    TotalOrderPrice=("TotalPrice", "sum")  # Soma os valores de cada pedido
).reset_index()

# **Calcular a última compra disponível**
ultima_compra = df_pedidos["InvoiceDate"].max()

# **Agrupar por Cliente e Calcular RFM**
rfm = df_pedidos.groupby("CustomerID").agg(
    Recency=("InvoiceDate", lambda x: (ultima_compra - x.max()).days),  # R
    Frequency=("InvoiceNo", "count"),  # F
    Monetary=("TotalOrderPrice", "mean")  # M (ticket médio)
).reset_index()

# **Visualizar o resultado**
print("Tabela RFM Calculada:")
print(rfm)

In [ ]:
rfm[["Recency", "Frequency", "Monetary"]].plot.box()

Ainda temos 2 grandes outliers em Monetary, vamos analisar os dois casos mais de perto para ver se faz sentido ignorá-los

In [ ]:
rfm.query("Monetary > 8000")

In [ ]:
df.query("CustomerID == 15098 or CustomerID == 15749")



---


O cliente 15749 não parece levantar nenhuma suspeita, deve ser um consumidor forte mesmo. O cliente 15098 parece ter sofrido um erro ou confusão de dados, veja que ele compra 60 unidades do item PICNIC BASKET WICKER SMALL, depois compra 60 unidades do item PICNIC BASKET WICKER 60 PIECES, o qual já inclui a quantidade na descrição. Vamos eliminar somente este cliente.

In [ ]:
rfm = rfm[rfm["CustomerID"] != 15098]
rfm

In [ ]:
rfm[["Recency", "Frequency", "Monetary"]].plot.box()

In [ ]:
rfm.describe()



---

Mesmo após remover os outliers mais notáveis, ainda podemos perceber uma diferença muito grande entre a maior parte dos dados (75%) e o valor máximo.
Para tentar amenizar isso, vamos usar um método de clip


In [ ]:
rfm_clip = rfm.apply(lambda x: x.clip(upper = x.quantile(.95)))
rfm_clip.describe()

In [ ]:
from sklearn.preprocessing import scale
rfm_clip_scaled = rfm_clip.apply(scale)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, DBSCAN, MeanShift, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from yellowbrick.cluster import kelbow_visualizer

In [ ]:
kelbow_visualizer(KMeans(), rfm_clip_scaled, k = 10, timings = False)

In [ ]:
# Create list of dictionaries

cluster_metrics = silhouette_score, davies_bouldin_score, calinski_harabasz_score
cluster_metrics_results = []
X = rfm_clip_scaled.copy()

for k in range(2,11):
  model = KMeans(n_clusters=k, random_state=0)
  labels = model.fit_predict(X)
  cluster_results_dict = {'k': k}
  cluster_results_dict['inertia'] = model.inertia_
  for metric in cluster_metrics:
    cluster_results_dict[metric.__name__] = metric(X, labels)
  cluster_metrics_results.append(cluster_results_dict)

pd.DataFrame(cluster_metrics_results).set_index('k').style.background_gradient()

In [ ]:
kmeans = KMeans(4)
kmeans_labels = kmeans.fit_predict(rfm_clip_scaled)



---

Vamos usar K = 4, mesmo o kelbow tendo sugerido 5, pois ao final, seriam gerados dois grupos muito parecidos

In [ ]:
import plotly.express as px
px.scatter_3d(rfm_clip, x = "Recency", y = "Frequency", z = "Monetary", color = kmeans_labels.astype(str))

In [ ]:
kelbow_visualizer(AgglomerativeClustering(), rfm_clip_scaled, k=10, timings=False)

In [ ]:
# Create list of dictionaries

cluster_metrics = silhouette_score, davies_bouldin_score, calinski_harabasz_score
cluster_metrics_results = []
X = rfm_clip_scaled.copy()

for k in range(2,11):
  model = AgglomerativeClustering(n_clusters=k)
  labels = model.fit_predict(X)
  cluster_results_dict = {'k': k}
  for metric in cluster_metrics:
    cluster_results_dict[metric.__name__] = metric(X, labels)
  cluster_metrics_results.append(cluster_results_dict)

pd.DataFrame(cluster_metrics_results).set_index('k').style.background_gradient()

In [ ]:
hclust = AgglomerativeClustering(n_clusters=4)
hclust_labels = hclust.fit_predict(rfm_clip_scaled)

In [ ]:
px.scatter_3d(rfm_clip, x = "Recency", y = "Frequency", z = "Monetary", color = hclust_labels.astype(str))

In [ ]:
# Select best model by covariance type and K (Practical Statistics for Data Scientists)
results = []
k_range = range(1,9)
covariance_types = ['full', 'tied', 'diag', 'spherical']
X = rfm_clip_scaled.copy()

for n_components in k_range:
  for covariance_type in covariance_types:
    mclust = GaussianMixture(n_components=n_components, n_init=10, warm_start=True, covariance_type=covariance_type)
    mclust.fit(X)
    results.append({
      'bic': mclust.bic(X),
      'n_components': n_components,
      'covariance_type': covariance_type,
    })

results = pd.DataFrame(results)
results.sort_values('bic').head()

In [ ]:
sns.lineplot(data = results, x = 'n_components', y='bic', hue='covariance_type');

In [ ]:
mclust = GaussianMixture(n_components=5, covariance_type='full', n_init=10)
mclust_labels = mclust.fit_predict(rfm_clip_scaled)

In [ ]:
px.scatter_3d(rfm_clip, x='Recency', y='Frequency', z='Monetary', color=mclust_labels.astype(str))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
centers = pd.DataFrame(kmeans.cluster_centers_, columns=rfm_clip_scaled.columns)

In [ ]:
# Adaptado do livro "Practical Statistics for Data Scientists"

# Remova 'CustomerID' de 'centers' antes de plotar
centers = centers.drop(columns=['CustomerID'])

# Encontrar o maior valor absoluto em todos os clusters
maxPC_global = 1.01 * centers.abs().max().max()  # O maior valor absoluto no DataFrame

fig,axes = plt.subplots(nrows=4, figsize=(14,12), sharex=True)

for i,ax in enumerate(axes):
  center = centers.loc[i,:]
  maxPC = 1.01 * center.abs().max()
  colors = ['green' if l > 0 else 'red' for l in center]
  center.plot.bar(ax=ax, color=colors)
  ax.set_ylabel(f'Cluster {i+1}')
  # Aplicar o mesmo limite para todos os eixos Y
  ax.set_ylim(-maxPC_global, maxPC_global)
  ax.axhline(color='gray')
  ax.xaxis.set_ticks_position('none')

plt.xticks(rotation=60, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
(
  rfm_clip.assign(cluster=kmeans_labels)
  .groupby('cluster')
  .mean()
  .transpose()
  .style.background_gradient(cmap='YlOrRd', axis=1)
)



---
Identificamos 4 grupos de clientes bem distintos:

1.   Clientes deste cluster são os mais recentes e frequentes. Eles compram regularmente e têm um gasto médio razoável. Estes clientes podem ser considerados fiéis, que fazem compras repetidas e têm um bom histórico de gastos.
2.   Este grupo de clientes fez compras há algum tempo (cerca de 1 mês e meio atrás), mas tem uma frequência mais baixa de compras. O gasto médio também é um pouco menor, mas ainda substancial. Esses clientes podem estar em um estágio intermediário: não são recentes, mas também não estão completamente inativos.
3. Este grupo de clientes fez compras ainda mais distantes, com uma frequência também moderada. No entanto, eles se destacam por gastar mais por compra, o que pode indicar que são clientes menos frequentes, mas que compram em maiores quantidades. Eles são um grupo importante devido ao seu alto valor monetário.
4. Este cluster representa clientes inativos há muito tempo (mais de 8 meses). A frequência de compras é muito baixa, e o gasto também é relativamente baixo. Esses clientes podem ser considerados como clientes em risco de abandono, churned, ou clientes esporádicos, que não geraram compras recentes e provavelmente não estão muito envolvidos com a marca.

Com isso, pode ser elaborada uma estratégia baseada em cada grupo individualmente, ou conforme a empresa desejar. Por exemplo, os grupos 2 e 3 representam clientes em risco, que não compram com tanta frequência quanto o primeiro grupo. Assim, campanhas de reativação, como ofertas especiais ou lembretes de produtos, podem ser direcionadas para tentar trazer esses clientes de volta.
Ainda, o grupo 2 tem um alto gasto médio, o que indica que, mesmo com baixa frequência, eles podem ser clientes importantes para a marca. Estratégias de up-selling ou cross-selling podem ser eficazes, oferecendo produtos complementares de alto valor.
